In [30]:
# Imports
import os 
from predictionguard import PredictionGuard
import pandas as pd 
import json
from langchain.prompts import PromptTemplate, FewShotPromptTemplate
import numpy as np
from torch import nn
from skorch import NeuralNetClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix, roc_curve, auc, RocCurveDisplay, ConfusionMatrixDisplay, accuracy_score, classification_report
import matplotlib.pyplot as plt
from torch.optim import Adam
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import f1_score, classification_report
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImbPipeline
import time

In [31]:
# Grief Journey Map
JourneyMap = {
    "1. Immediate Grief, Shock, and Emotion": "Overwhelmed, loss of purpose; shock and trauma emotions (isolation) present and challenging to understand. Individuals may struggle to deal with family responsibilities alone. Surviving Child: Feeling disconnected without guidance and attention from grieving adults.", 

    "2. Navigating Family Relationships": "Experiencing tension between individuals within the family unit; lack of support from family members. Surviving Family Unit: Perception of other family members’ grief experience. Each family member may be at different phases of their grief journey.",

    "3. Learning to Process Grief": "Experiencing grief and learning to process those emotions. Surviving Child: Seeks guidance and acknowledgment of grief; benefit from opportunities to open up and process with kids in similar situations to normalize emotions.", 

    "4. Moments That Matter": "Renewed experience of grief around anniversaries of loss, holidays, and special moments. Surviving Family Unit: Navigating special moments (sports, school achievements, moments that matter).", 

    "5. Feeling Immersed, Connected & Seen": "Finding new purpose and goals to begin moving towards Positive Integration. Surviving Family Unit: Connected to a broader community; support system; not the only person/family experiencing loss.", 

    "6. New Growth & Purpose": "Healthy point in grief journey; feeling capable to help others and a desire to do so. Surviving Family Unit: Ready to give back to the TAPS community through mentorship programs, volunteering at charity drives & events, etc." 
    }

dict = {
    "Stage Number": [int(key.split('.')[0]) for key in JourneyMap.keys()],
    "Stage Name": [key.split('. ', 1)[1] for key in JourneyMap.keys()],
    "Description": JourneyMap.values()
}

JourneyMapDF = pd.DataFrame(dict)
JourneyMapDF.set_index("Stage Number", inplace=True)
JourneyMapDF.head()

,Stage Name,Description
Stage Number,,
1,"Immediate Grief, Shock, and Emotion","Overwhelmed, loss of purpose; shock and trauma..."
2,Navigating Family Relationships,Experiencing tension between individuals withi...
3,Learning to Process Grief,Experiencing grief and learning to process tho...
4,Moments That Matter,Renewed experience of grief around anniversari...
5,"Feeling Immersed, Connected & Seen",Finding new purpose and goals to begin moving ...


In [32]:
# Survey Questions
SurveyQuestions = [
    {
        "question": "Please share all of the reasons you chose to attend the TAPS [SEMINAR NAME]",
        "options": [
            "I had attended a previous Regional or Nationals seminar",
            "To connect with other survivors who share a similar loss",
            "To learn more about resources TAPS has to offer",
            "To learn new tools and information to help me with my grief",
            "To learn more about how to support my adult family members in their grief",
            "To learn more about how to support my child(ren) in their grief",
            "For my child(ren) to attend Good Grief Camp",
            "For my child(ren) to connect with a Military Mentor"
        ],
        "type": "multiple_choice"
    },
    {
        "question": "How did you find out about the TAPS [SEMINAR NAME]?",
        "options": [
            "I found this event while searching for grief resources",
            "I attended a seminar last year and had already marked my calendar!",
            "TAPS invited me to this event via email",
            "My TAPS Survivor Care Team Member invited me",
            "My Peer Mentor or another survivor invited me",
            "Through the TAPS website",
            "Through the TAPS magazine",
            "Through a TAPS Social Media Page"
        ],
        "type": "multiple_choice"
    },
    {
        "question": "This seminar helped me to feel socially connected",
        "options": ["Disagree", "Neither agree nor disagree", "Agree"],
        "type": "likert"
    },
    {
        "question": "This seminar taught me new ways to cope with my grief",
        "options": ["Disagree", "Neither agree nor disagree", "Agree"],
        "type": "likert"
    },
    {
        "question": "This seminar helped me to better understand my grief",
        "options": ["Disagree", "Neither agree nor disagree", "Agree"],
        "type": "likert"
    },
    {
        "question": "This seminar has given me hope for the future",
        "options": ["Disagree", "Neither agree nor disagree", "Agree"],
        "type": "likert"
    },
    {
        "question": "Based on your experience at the TAPS [SEMINAR NAME], how likely are you to continue to connect with TAPS virtually / at in-person programs?",
        "options": [
            "Extremely likely / Very likely",
            "Somewhat likely",
            "Neither likely nor unlikely / Neutral",
            "Somewhat unlikely",
            "Extremely unlikely / Not at all likely"
        ],
        "type": "likelihood"
    },
    {
        "question": "Based on your experience at the TAPS [SEMINAR NAME], how likely are you to continue to bring your child(ren) to connect with TAPS at in-person programs?",
        "options": [
            "Extremely likely / Very likely",
            "Somewhat likely",
            "Neither likely nor unlikely / Neutral",
            "Somewhat unlikely",
            "Extremely unlikely / Not at all likely"
        ],
        "type": "likelihood"
    },
    {
        "question": "I have a positive outlook toward life",
        "options": ["Strongly Disagree", "Disagree", "Agree", "Strongly Agree"],
        "type": "likert"
    },
    {
        "question": "I have short and/or long range goals",
        "options": ["Strongly Disagree", "Disagree", "Agree", "Strongly Agree"],
        "type": "likert"
    },
    {
        "question": "I feel all alone",
        "options": ["Strongly Disagree", "Disagree", "Agree", "Strongly Agree"],
        "type": "likert"
    },
    {
        "question": "I can see possibilities in the midst of difficulties",
        "options": ["Strongly Disagree", "Disagree", "Agree", "Strongly Agree"],
        "type": "likert"
    },
    {
        "question": "I have faith that gives me comfort",
        "options": ["Strongly Disagree", "Disagree", "Agree", "Strongly Agree"],
        "type": "likert"
    },
    {
        "question": "I feel scared about my future",
        "options": ["Strongly Disagree", "Disagree", "Agree", "Strongly Agree"],
        "type": "likert"
    },
    {
        "question": "I can recall happy/joyful times",
        "options": ["Strongly Disagree", "Disagree", "Agree", "Strongly Agree"],
        "type": "likert"
    },
    {
        "question": "I have deep inner strength",
        "options": ["Strongly Disagree", "Disagree", "Agree", "Strongly Agree"],
        "type": "likert"
    },
    {
        "question": "I am able to give and receive care/love",
        "options": ["Strongly Disagree", "Disagree", "Agree", "Strongly Agree"],
        "type": "likert"
    },
    {
        "question": "I have a sense of direction",
        "options": ["Strongly Disagree", "Disagree", "Agree", "Strongly Agree"],
        "type": "likert"
    },
    {
        "question": "I believe that each day has potential",
        "options": ["Strongly Disagree", "Disagree", "Agree", "Strongly Agree"],
        "type": "likert"
    },
    {
        "question": "I feel my life has value and worth",
        "options": ["Strongly Disagree", "Disagree", "Agree", "Strongly Agree"],
        "type": "likert"
    },
    {
        "question": "Please share with TAPS your favorite moment of the weekend? Did you have a breakthrough moment this weekend you would like to share?",
        "type": "free_text"
    },
    {
        "question": "Please share any additional feedback you have regarding your TAPS Seminar experience.",
        "type": "free_text"
    }
]

In [33]:
# Training Data Import
data = pd.read_csv("examples.csv")


In [34]:
stage_data = data[data["label"] == 1]
stage_data

,Id,q1,q2,q3,q4,q5,q6,q7,q8,q9,...,q14,q15,q16,q17,q18,q19,q20,q21,q22,label
6,eaec911b-a826-4695-8284-594ed05a0969,3,4.0,0,1,1,0,4,2,0,...,4,0,1,0,1,0,0,"""I attended the TAPS seminar this past weekend...",NaN,1
10,b3bce329-0cda-4b87-a86c-65d545cd4f60,1,3.0,1,1,2,1,1,1,0,...,3,1,0,0,0,1,0,"""During the weekend seminar, my favorite momen...",NaN,1
17,87cb0f7b-1b7f-4bcb-9e03-223f0619ac62,2,2.0,1,1,1,1,2,4,1,...,4,0,0,1,0,3,0,NaN,NaN,1
24,cbdaa24d-fe96-4bed-84fe-0a5d48655e01,1,2.0,0,1,2,0,3,4,1,...,3,1,1,0,0,0,0,NaN,"""As a surviving child, the TAPS Seminar proved...",1
27,e7717a7b-7a94-42fb-b57b-f5d7437b6ce2,3,4.0,1,2,2,1,2,3,1,...,4,1,1,1,0,1,0,NaN,NaN,1
35,430e001f-a95e-4afc-b297-15a020104c2e,1,NaN,1,2,2,1,1,1,1,...,4,0,1,1,0,1,0,"""Although it's been a challenging few days, I ...","At TAPS Seminar, I connected with fellow survi...",1
38,df79850f-84fc-44d0-bca4-b695cb37764e,3,1.0,1,1,1,1,2,2,0,...,3,0,0,0,0,2,1,NaN,NaN,1


In [35]:
# Function to create prompt with stage-specific examples
def specific_prompt(stage_number, stage_name, description, question_text, options_text, data):
    # Filter the data for the specific grief stage
    stage_data = data[data["label"] == stage_number]
    
    # Limit to a few examples to keep the prompt concise
    stage_examples = stage_data.sample(min(3, len(stage_data)))  # Get up to 3 examples

    # Format examples for the prompt
    example_texts = "\n\n".join(
        [
            f"Example {i+1}:\n" +
            "\n".join([f"Q{idx+1}: {row[f'q{idx+1}']}" for idx in range(20)]) + 
            (f"\nFree Text Response 1: {row['q21']}" if pd.notnull(row['q21']) else "") +
            (f"\nFree Text Response 2: {row['q22']}" if pd.notnull(row['q22']) else "")
            for i, row in stage_examples.iterrows()
        ]
    )

    # Structure the prompt messages for the LLM
    messages = [
        {
            "role": "system",
            "content": f"You are a person who has had a death in the military or veteran community (at some point in time). Your current stage of grief is {stage_name}: {description} You are asked by the Tragedy Assistance Program for Survivors (TAPS) to fill out this survey based on your current stage of grief. Only return the number that corresponds to your answer, nothing else. Only pick 1 number per question."
        },
        {
            "role": "user",
            "content": f"""
            {example_texts}
            
            Current Question: {question_text}
            Options: {options_text}
            Only respond with the number that corresponds to your answer, nothing else (for example: '1'). 
            Response:
            """
        }
    ]

    return messages


In [42]:
os.environ["PREDICTIONGUARD_API_KEY"] = "Oq62vYfSJRwjnFQcUnJy5PM3SRVejYtJCXWSxnfv"
client = PredictionGuard()

responses_df = pd.DataFrame(columns=data.columns)

for run_id in range(20):
    for stage, description in JourneyMap.items():
        stage_number = int(stage.split('.')[0])
        stagename = stage.split('.', 1)[1]
        responses = {"Id": f"{run_id}_{stagename}", 
                     "label": stage_number}
        
        for question_index, question_data in enumerate(SurveyQuestions[0:20]):
            question_text = question_data["question"]
            options = question_data["options"]
            options_text = "\n".join([f"{i}. {option}" for i, option in enumerate(options)])

            # Use the specific_prompt function to generate messages with stage-specific examples
            messages = specific_prompt(
                stage_number=stage_number,
                stage_name=stagename,
                description=description,
                question_text=question_text,
                options_text=options_text,
                data=data
            )

            # Call the LLM with the generated messages
            result = client.chat.completions.create(
                model="neural-chat-7b-v3-3",
                messages=messages, 
                temperature=0.8, 
                max_tokens=2
            )
            answer = result["choices"][0]["message"]["content"].strip()
            responses[f"q{question_index+1}"] = answer
            time.sleep(1)
        
        # Add placeholders for free-text responses if needed
        responses["q21"] = ''
        responses["q22"] = ''

        # Append the generated responses to the DataFrame
        responses_df = pd.concat([responses_df, pd.DataFrame([responses])], ignore_index=True)


In [40]:
responses_df

,Id,q1,q2,q3,q4,q5,q6,q7,q8,q9,...,q14,q15,q16,q17,q18,q19,q20,q21,q22,label
0,"0_ Immediate Grief, Shock, and Emotion",1,3,2,2,2,2,2,3,0,...,3,1,1,1,1,2,0,,,1
1,0_ Navigating Family Relationships,1,3,2,2,2,2,2,2,1,...,2,0,3,2,1,2,2,,,2
2,0_ Learning to Process Grief,1,2,2,2,2,2,1,1,1,...,2,2,2,1,2,2,2,,,3
3,0_ Moments That Matter,1,0,2,2,2,2,3,2,2,...,3,1,3,2,0,2,2,,,4
4,"0_ Feeling Immersed, Connected & Seen",1,3,2,2,2,2,7,2,2,...,2,2,3,2,2,2,2,,,5
5,0_ New Growth & Purpose,1,5,2,2,2,2,0,2,3,...,Q1,3,3,2,3,2,3,,,6
6,"1_ Immediate Grief, Shock, and Emotion",1,2,2,2,2,2,1,3,0,...,2,1,2,1,0,2,1,,,1
7,1_ Navigating Family Relationships,1,5,2,2,2,2,2,3,1,...,2,2,3,2,2,2,2,,,2
8,1_ Learning to Process Grief,1,1,2,2,2,2,2,1,2,...,2,2,3,2,2,2,2,,,3
9,1_ Moments That Matter,1,5,2,2,2,2,1,2,2,...,2,1,1,2,2,2,2,,,4


In [41]:
responses_df.to_csv('surprisemebrother_60.csv', index=False)